In [1]:
import numpy as np
from collections import deque, defaultdict
import math
import sys
import heapq
import time
from tqdm.notebook import tqdm

def timeit(action_str, print_time=True):
    def inner_function(func):
        def wrapper(*arg, **kwarg):
            start_time = time.time()
            print(action_str.center(100, "="))
            res = func(*arg, **kwarg)
            t = int(time.time() - start_time)
            m, s = t // 60, t % 60
            print(
                (" {}: Done ".format(action_str) 
                 + print_time * "in {:0>2.0f}:{:0>2.0f}s ".format(m, s)
                ).center(100, "=")
            )
            return res
        return wrapper
    return inner_function

## Week 1: Graph search and connectivity

In [75]:
@timeit(" Loading graph file ")
def load_graph(path='./SSC.txt'):
    return np.loadtxt(path, dtype=int)

G_np = load_graph('./SCC.txt')    

======================================== Loading graph file ========================================
===============================  Loading graph file : Done in 00:33s ===============================


In [16]:
class GraphUtils():
    def __init__(self, adj_list=None):
        if adj_list is not None:
            self.build_graph(adj_list)
            self.n = max(self.graph.keys())
        else:
            self.graph = defaultdict(list)
            self.n = 0
        
    def add_edge(self, u, v):
        if u not in self.graph.keys():
            self.n += 1
        self.graph[u].append(v)
    
    @timeit(action_str=" Building graph ")
    def build_graph(self, adj_list):
        self.graph = defaultdict(list)
        for u, v in adj_list:
            self.graph[u].append(v)
            self.graph[v]
#         print(max(self.graph.keys()))
    
    @timeit(action_str=" Transposing graph ")
    def get_transposed(self):
        g = Graph(adj_list=None)
        for u in self.graph: 
            for v in self.graph[u]: 
                g.graph[v].append(u)
                g.graph[u]
        return g        


### Breadth-first search

In [41]:
class Graph(GraphUtils):
    def __init__(self, adj_list):
        super().__init__(adj_list=adj_list)
        
    def BFS(self, s: int) -> None:
        explored = [False] * self.n
        Q = deque([s])
        explored[s - 1] = True
        while Q:
            v = Q.popleft()
            for w in self.graph[v]:
                if not explored[w - 1]:
                    explored[w - 1] = True
                    Q.append(w)

    def shortest_path(self, s: int) -> dict:
        """ BFS based shortest path """
        explored = [False] * self.n
        dist  ={x: (float('inf') if x != s else 0) for x in self.graph.keys()}
        Q = deque([s])
        while Q:
            v = Q.popleft()
            for w in self.graph[v]:
                if not explored[w - 1]:
                    explored[w - 1] = True
                    dist[w] = dist[v] + 1
                    Q.append(w)
        return dist

    def connectivity(self) -> list:
        """ BFS based undirected connectivity components """
        explored = [False] * self.n
        connected_components = []
        for s in self.graph.keys():
            if not explored[s - 1]:
                # start BFS
                component = [s]
                Q = deque([s])
                explored[s - 1] = True
                while Q:
                    v = Q.popleft()
                    for w in self.graph[v]:
                        if not explored[w - 1]:
                            explored[w - 1] = True
                            component.append(w)
                            Q.append(w)
                connected_components.append(component)
        return connected_components
            

In [36]:
g = Graph(G_np)
g.shortest_path(1)

========================================== Building graph ==========================================
=================================  Building graph : Done in 00:07s =================================


{1: 1,
 2: 2,
 3: 2,
 5: 2,
 6: 2,
 7: 2,
 8: 2,
 9: 3,
 10: 3,
 11: 3,
 12: 3,
 13: 3,
 14: 3,
 15: 3,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 3,
 22: 3,
 23: 3,
 24: 3,
 25: 3,
 26: 3,
 27: 3,
 28: 3,
 29: 3,
 30: 3,
 31: 3,
 32: 3,
 33: 3,
 34: 3,
 35: 3,
 36: 3,
 38: 10,
 40: 9,
 41: 8,
 42: 9,
 43: 9,
 44: 9,
 45: 9,
 46: 9,
 47: 9,
 48: 9,
 49: 10,
 50: 10,
 51: 10,
 52: 10,
 53: 10,
 54: 10,
 55: 8,
 56: 10,
 57: 10,
 58: 10,
 59: 10,
 60: 10,
 61: 10,
 62: 10,
 63: 10,
 64: 10,
 65: 10,
 66: 10,
 67: 10,
 68: 10,
 69: 9,
 70: 9,
 71: 9,
 72: 9,
 73: 9,
 74: 9,
 75: 9,
 76: 9,
 77: 9,
 78: 9,
 79: 9,
 80: 9,
 81: 9,
 82: 9,
 83: 9,
 84: 9,
 85: 9,
 86: 9,
 87: 9,
 88: 9,
 89: 9,
 90: 9,
 91: 9,
 92: 9,
 93: 10,
 94: 10,
 95: 10,
 96: 10,
 97: 10,
 98: 10,
 99: 10,
 100: 10,
 101: 10,
 102: 10,
 103: 10,
 104: 10,
 105: 10,
 106: 10,
 107: 10,
 109: 10,
 110: 10,
 111: 10,
 112: 10,
 113: 10,
 114: 10,
 115: 10,
 116: 10,
 117: 10,
 118: 10,
 119: 7,
 120: 10,
 121: 10,
 122: 11,
 1

### Depth first search

In [18]:
class Graph(GraphUtils):
    def __init__(self, adj_list):
        super().__init__(adj_list=adj_list)
    
    def topo_sort_rec(self) -> dict:
        """ Returns topological ordering labels """
        self.explored = [False] * self.n
        self.current_label = self.n
        self.labels = defaultdict(int)
        for s in self.graph.keys():
            if not self.explored[s - 1]:
                self.__DFS_rec_helper(s)
        return self.labels
    
    def __DFS_rec_helper(self, s: int) -> None:
        """ Recursive DFS for topological sort"""
        self.explored[s - 1] = True
        for v in self.graph[s]:
            if not self.explored[v - 1]:
                self.__DFS_rec_helper(v)
                self.labels[v] = self.current_label
                self.current_label -= 1
    
    def topo_sort_iter(self):
        explored = [0] * self.n
        stack = deque()
        order = deque()
        for s in reversed(list(self.graph.keys())):
            if not explored[s - 1]:
                stack.append(s)
                while stack:
                    u = stack[-1]
                    if explored[u - 1]:
                        u = stack.pop()
                        if explored[u - 1] == 1:
                            order.append(u)
                            explored[u - 1] = 2
                    else:
                        explored[u - 1] = 1
                        for v in self.graph[u]:  
                            if not explored[v - 1]:  
                                stack.append(v)
        return order
                    
        

In [19]:
g = Graph(G_np)
a = g.topo_sort_iter()
a

========================================== Building graph ==========================================
=================================  Building graph : Done in 00:08s =================================


deque([75219,
       370361,
       779428,
       370353,
       370345,
       370358,
       99896,
       99894,
       99907,
       99897,
       99906,
       99905,
       99904,
       99903,
       99902,
       99901,
       99893,
       99892,
       99900,
       99899,
       99898,
       99895,
       582217,
       606047,
       438475,
       830932,
       549896,
       606043,
       582216,
       429631,
       429628,
       429629,
       429630,
       582215,
       781668,
       582214,
       582213,
       576445,
       576444,
       576443,
       334284,
       334276,
       8335,
       704507,
       704512,
       704511,
       704510,
       31855,
       31854,
       320335,
       583377,
       28853,
       159094,
       628602,
       385916,
       385917,
       159101,
       385915,
       159100,
       159095,
       159102,
       159096,
       489481,
       701746,
       489480,
       701745,
       701744,
       701743,
  

### Strong components algorithm: Kosaraju

In [2]:
class Graph(GraphUtils):
    def __init__(self, adj_list):
        super().__init__(adj_list=adj_list)
        
    def __DFS_fill_order(self, s, stack):
        """ Recursive DFS for getting order of vertices """
        self.explored[s - 1] = True
        print(s)
        for v in self.graph[s]:
            if not self.explored[v - 1]:
                stack = self._DFS_fill_order(v, stack)
        stack.append(s)
        return stack
                
    def __DFS_scc(self, s):
        """ Recursive DFS for computing SCCs on reversed graph"""
        self.explored[s - 1] = True
        self.scc[-1].append(s)
        for v in self.graph[s]:
            if not self.explored[v - 1]:
                self._DFS_scc(v)
    
    @timeit(" Running Strongly Connected Components ")
    def SCC(self):
        # same as topo iter
        explored = [0] * self.n
        stack = deque()
        order = deque()
        for s in reversed(list(self.graph.keys())):
            if not explored[s - 1]:
                stack.append(s)
                while stack:
                    u = stack[-1]
                    if explored[u - 1]:
                        u = stack.pop()
                        if explored[u - 1] == 1:
                            order.append(u)
                            explored[u - 1] = 2
                    else:
                        explored[u - 1] = 1
                        for v in self.graph[u]:  
                            if not explored[v - 1]:  
                                stack.append(v)

        g_rev = self.get_transposed()
        explored = [False] * self.n
        self.scc = []
        stack = deque()
        while order:
            s = order.pop()
            if not explored[s - 1]:
                stack.append(s)
                component = []
                while stack:
                    u = stack.pop()
                    if not explored[u - 1]:
                        explored[u - 1] = True
                        component.append(u)
                    for v in g_rev.graph[u]:  # looking at reverse graph
                        if not explored[v - 1]:
                            stack.append(v)
                self.scc.append(component)
        return self.scc
    

NameError: name 'GraphUtils' is not defined

In [15]:
g = Graph(G_np)
# for adj_list in cases[4:5]:
#     g = Graph(adj_list)
#     print(g.graph)
scc = g.SCC()
print(sorted(map(lambda x: len(set(x)), scc), reverse=True))

========================================== Building graph ==========================================
875714
=================================  Building graph : Done in 00:07s =================================
============================== Running Strongly Connected Components ===============================
======================================== Transposing graph =========================================
===============================  Transposing graph : Done in 00:03s ================================
=====================  Running Strongly Connected Components : Done in 00:18s ======================
[434821, 968, 459, 313, 211, 205, 197, 177, 162, 152, 149, 146, 138, 125, 122, 112, 102, 102, 101, 97, 96, 93, 90, 85, 81, 78, 78, 77, 77, 77, 75, 73, 72, 72, 72, 71, 71, 70, 69, 69, 68, 68, 68, 67, 67, 66, 65, 65, 64, 64, 63, 63, 59, 59, 59, 59, 59, 58, 58, 56, 56, 56, 55, 55, 55, 55, 55, 55, 54, 54, 54, 53, 53, 53, 53, 52, 52, 51, 51, 51, 51, 51, 49, 49, 49, 48, 48, 48, 48, 47, 47, 

## Week 2: Dijkstra's shortest path algorithm

BFS solves the shortest path problem when $\forall \, e \in E, \; len(e) =1$. Dijkstra is more general.

Complexity:
- n vertices, m edges
- stoping when all vertices have been visited and all adges have been explored: $O(mn)$
- with heap for the find min dist node subroutine, can speed up to $O(m \log(n))$

In [2]:
@timeit(" Loading graph file ")
def open_dijkstra(path='./Dijkstra.txt'):
    with open(path) as file:
        G_lst = []
        for row in file:
            row = row.split()
            G_lst.append(row)
    return G_lst
        
G_lst = open_dijkstra()

======================================== Loading graph file ========================================
===============================  Loading graph file : Done in 00:00s ===============================


In [3]:
class Graph():
    def __init__(self, adj_list=None):
        self.build_graph(adj_list)
        self.n = max(self.graph.keys())
    
    @timeit(" Building graph ")
    def build_graph(self, adj_list):
        self.graph = defaultdict(list) 
        for row in adj_list:
            key = int(row[0])
            for edge in row[1:]:
                node, dist = map(int, edge.split(','))
                self.graph[key].append((node, dist))
                self.graph[node]
    
    @timeit( "Transposing graph ")
    def get_transposed(self):
        g = Graph()
        # Recur for all the vertices adjacent to this vertex 
        for u in self.graph: 
            for v in self.graph[i]: 
                g.graph[u].append(v)
        return g 
    
    def dijkstra(self, s):
        explored = defaultdict(bool)
        dist = {key: float('inf') for key in self.graph.keys()}
        dist[s] = 0 
        spt = defaultdict(list)
        for _ in self.graph.keys():
            # looking for closest vertex
            min_dist = float('inf')
            for v in self.graph.keys(): 
                if dist[v] < min_dist and not explored[v]: 
                    min_dist = dist[v] 
                    u = v
            # u is always s in first iteration
            explored[u] = True
            
            for v, d in self.graph[u]:
                if not explored[v] and dist[v] > dist[u] + d:
                    dist[v] = dist[u] + d
                    spt[v] = spt[u] + [u]
        return dist, spt
    
    def dijkstra_heap(self, s):
        explored = defaultdict(bool)
        dist = {key: float('inf') for key in self.graph.keys()}
        dist[s] = 0 
        spt = defaultdict(list)
        q = [(0, s)]  # dist, node, pred
        for _ in self.graph.keys():
            # looking for closest vertex
            _, u = heapq.heappop(q)
            explored[u] = True
            for v, d in self.graph[u]:
                if not explored[v] and dist[v] > dist[u] + d:
                    dist[v] = dist[u] + d
                    spt[v] = spt[u] + [u]
                    heapq.heappush(q, (dist[v], v))
        return dist, spt
        

In [4]:
res = []
g = Graph(G_lst)
d, p = g.dijkstra_heap(1)
for i in [7,37,59,82,99,115,133,165,188,197]:
    res.append(str(d[i]))
res = ",".join(res)
print(res)

========================================== Building graph ==========================================
=================================  Building graph : Done in 00:00s =================================
2599,2610,2947,2052,2367,2399,2029,2442,2505,3068


In [8]:
@timeit(" Comparing running times ")
def compare_times(G_lst: list, n: int = 100):
    """
    Takes graph and sampling time n, and compute dijkstra for all nodes to all nodes n times
    returns avg time per iter
    """

    def compare_times_(func):
        print(" Running {} ".format(func.__name__).center(100, "-"))
        start_time = time.time()
        for _ in tqdm(range(n), total=n):
            for i in range(1, g.n):
                func(i)
        res_time = (time.time() - start_time) / n
        if res_time >= 10:
            m, s = res_time // 60, res_time % 60
            t = "{:0>2}min:{:0>2}s".format(m, s)
        else:
            t = "{:.0f}ms".format(res_time * 1000)
        print(
            "Avg running time for {}: {}".format(func.__name__, t)
        )

    g = Graph(G_lst)
    for func in [g.dijkstra, g.dijkstra_heap]:
        compare_times_(func)
        
compare_times(G_lst, n=20)

===================================== Comparing running times ======================================
========================================== Building graph ==========================================
=================================  Building graph : Done in 00:00s =================================
----------------------------------------- Running dijkstra -----------------------------------------



Avg running time for dijkstra: 855ms
-------------------------------------- Running dijkstra_heap ---------------------------------------



Avg running time for dijkstra_heap: 216ms
============================  Comparing running times : Done in 00:21s =============================


## Week 3: Heaps and binary search trees

In [66]:
class MedianMaintenance():
    def __init__(self, nums=[]):
        self.nums = nums
        self.heap_low = []
        self.heap_high = []
        self.same_heap_length = True
        
    def insert(self, num):
        self.nums.append(num)
        if not self.heap_low:
            heapq.heappush(self.heap_low, - num)
            self.same_heap_length = False
        elif num <= - self.heap_low[0]:  # below or equal to median
            if self.same_heap_length == True:
                heapq.heappush(self.heap_low, -num)
                self.same_heap_length = False
            else:  # too much elements in heap_low
                new_num = - heapq.heappushpop(self.heap_low, - num)
                heapq.heappush(self.heap_high, new_num)
                self.same_heap_length = True
        else:  # above median
            if self.same_heap_length == True:  # heap_low must always contain median
                new_num = heapq.heappushpop(self.heap_high, num)
                heapq.heappush(self.heap_low, - new_num)
                self.same_heap_length = False
            else:  # complete heap_high
                heapq.heappush(self.heap_high, num) 
                self.same_heap_length = True

    def median(self):
        median = - self.heap_low[0]
        return median

In [3]:
data = np.loadtxt("./median.txt", dtype=int)

In [134]:
medians = []
verif = []
m = MedianMaintenance(nums=[])
for num in tqdm(data, total=len(data)):
    m.insert(num)
    medians.append(m.median())
    verif.append(sorted(m.nums)[(len(m.nums)+1) // 2 - 1])
    if medians[-1] != verif[-1]:
        print(sorted(m.nums))
        print(medians[-1])
        print(verif[-1])
        print(m.heap_high, m.heap_low)
        break
        
sum(medians) % 10000

1213

## Week 4: hash tables and the 2-sum problem  

In [129]:
@timeit(action_str = " Running Two sum alg ")
def two_sum(array, targets=(-10000, 10000)):
    H = defaultdict(set)
    res = set()
    print(" Hashing ".center(100, "-"))
    for x in tqdm(array):
        H[abs(x) // targets[1]].add(x)
    print(" Searching for targets ".center(100, "-"))
    for x in tqdm(array):
        h = abs(x) // targets[1]
        for y in H[h - 1].union(H[h]).union(H[h + 1]):
            if targets[0] <= x + y <= targets[1]:
                res.add(x+y)
    return len(res)

In [85]:
A = np.loadtxt("./2_sum.txt", dtype=float)

In [133]:
two_sum(A, targets=(-10000, 10000))

======================================= Running Two sum alg ========================================
--------------------------------------------- Hashing ----------------------------------------------



-------------------------------------- Searching for targets ---------------------------------------



==============================  Running Two sum alg : Done in 00:31s ===============================


42760